In [1]:
import sympy as sp

import sys,os,subprocess
sys.path.insert(0,'/work/JAM/apps/bin')
os.environ["PATH"]='/work/JAM/apps/bin:'+os.environ["PATH"]
from qgraf import QGRAF
from form import FORM,to_sympy,run_form,run_form2,to_sympy
from tools import checkdir,save,load,lprint,lpprint,evalf

import sympy as sp
from sympy.parsing.sympy_parser import parse_expr
from sympy import mathematica_code as mcode
from sympy.utilities.lambdify import lambdastr

# Derive Cross Section

In [2]:
code = '''
Off Statistics;
Symbols M,Q,lam,E,h;
Symbols x,y;
Symbols n;
Vectors P,q,S;
Vectors Phat,Shat;
Vectors l,lp;
Autodeclare Vector V;
Tensor ghat;
Autodeclare Index o;
Autodeclare Symbols F,g;
Cfunction dot,e(antisymmetric);

L [L] = 2*(l(o1)*lp(o2) + lp(o1)*l(o2) - l.lp*d_(o1,o2) - I_*lam*e(o1,o2,o3,o4)*l(o3)*lp(o4));
L [W] = I_*e(o1,o2,o5,o6)*q(o5)*(S(o6)*g1 + (S(o6) - S.q/P.q*P(o6))*g2)/P.q
        +(((P(o1) - P.q/q^2*q(o1))*(S(o2) - S.q/q^2*q(o2)) + (S(o1) - S.q/q^2*q(o1))*(P(o2) - P.q/q^2*q(o2)))/2 
            - S.q/P.q*(P(o1) - P.q/q^2*q(o1))*(P(o2) - P.q/q^2*q(o2)))*g3/P.q
        + S.q/P.q*((P(o1) - P.q/q^2*q(o1))*(P(o2) - P.q/q^2*q(o2))*g4/P.q -(d_(o1,o2) - q(o1)*q(o2)/q^2)*g5);

L [LW] = [L]*[W];

*contract;
id e(o1,o2,o3?,o4?)*e(o1,o2,o5?,o6?) = -2*(d_(o3,o5)*d_(o4,o6) - d_(o3,o6)*d_(o4,o5));
id lp = l - q;
id P.S = 0;
id P.P = M^2;
id S.l = -h*E*M;*Q^2/2/x/y;
id S.q = -h*E*M*(y+Q^2/2/E^2);*-h*Q^2/2/x*(1+2*x^2*y*M^2/Q^2);
id E = Q^2/2/M/x/y;
id E^n? = (Q^2/2/M/x/y)^n;
id E^-n? = (Q^2/2/M/x/y)^-n;
id l.l = 0;
id q.q = -Q^2;
id P.l = P.q/y;
id P.q = Q^2/2/x;
id P.q^-1 = 2*x/Q^2;
id q.l = -Q^2/2;
id V1?.V2? = dot(V1,V2);
id V1?.V2?^n? = (dot(V1,V2))^n;
id V1?.V2?^-n? = (dot(V1,V2))^-n;

print [LW];
'''
data = run_form(code,verb=0)
print(data['LW'])
data['LW'] = data['LW'].replace(r'Q','q').replace('E','e').replace('S','s')
data['LW'] = data['LW'].replace(r'sqrt_','sqrt')
data['LW'] = to_sympy(data['LW'])

lam  = sp.symbols(r'\lambda')
h    = sp.symbols(r'h')
M    = sp.symbols(r'M')
Q    = sp.symbols(r'Q')
E    = sp.symbols(r'E')
x    = sp.symbols(r'x')
y    = sp.symbols(r'y')
alfa = sp.symbols(r'\alpha')
F1,F2,F3       = sp.symbols('F_1 F_2 F_3')
g1,g2,g3,g4,g5 = sp.symbols('g_1 g_2 g_3 g_4 g_5')


data['LW'] = data['LW'].subs(r'lam',lam)
data['LW'] = data['LW'].subs('q',Q).subs('e',E)
data['LW'] = data['LW'].subs(sp.Symbol('F1'),F1).subs(sp.Symbol('F2'),F2).subs(sp.Symbol('F3'),F3)
data['LW'] = data['LW'].subs(sp.Symbol('g1'),g1).subs(sp.Symbol('g2'),g2).subs(sp.Symbol('g3'),g3).subs(sp.Symbol('g4'),g4).subs(sp.Symbol('g5'),g5)

- 2*Q^2*h*x^-1*y^-2*g4 + 2*Q^2*h*x^-1*y^-1*g4 - 2*Q^2*h*g5 - 4*Q^2*lam*h*y^-1*g1 + 2*Q^2*lam*h*g1 - 4*M^2*h*x*y^-1*g4 + 4*M^2*h*x*y^-1*g3 + 6*M^2*h*x*g4 - 4*M^2*h*x*g3 - 4*M^2*h*x^2*y*g5 + 8*M^2*lam*h*x^2*g2 + 4*M^2*lam*h*x^2*y*g1 + 4*M^4*Q^-2*h*x^3*y*g4 - 4*M^4*Q^-2*h*x^3*y*g3


In [3]:
pol = (data['LW'].subs(h,-1) - data['LW'].subs(h,1))#/sp.S(2)

factor1  = 2*sp.pi*y*alfa**2/Q**4
factor2 = 8*sp.pi*alfa**2/x/y/Q**2

factor = factor1/factor2

M_Q = sp.Symbol(r'\left(\frac{M}{Q}\right)')

coeff4 = factor*pol.subs(g1,1).subs(g2,0).subs(g3,0).subs(g4,0).subs(g5,0).subs(M,Q*M_Q)
coeff5 = factor*pol.subs(g1,0).subs(g2,1).subs(g3,0).subs(g4,0).subs(g5,0).subs(M,Q*M_Q)
coeff6 = factor*pol.subs(g1,0).subs(g2,0).subs(g3,1).subs(g4,0).subs(g5,0).subs(M,Q*M_Q)
coeff7 = factor*pol.subs(g1,0).subs(g2,0).subs(g3,0).subs(g4,1).subs(g5,0).subs(M,Q*M_Q)
coeff8 = factor*pol.subs(g1,0).subs(g2,0).subs(g3,0).subs(g4,0).subs(g5,1).subs(M,Q*M_Q)

display(coeff4.simplify())
display(coeff5.simplify())
display(coeff6.simplify())
display(coeff7.simplify().factor(x,y))
display(coeff8.simplify().factor(x,y))

\lambda*x*y*(-y*(2*\left(\frac{M}{Q}\right)**2*x**2*y + 1) + 2)

-4*\lambda*\left(\frac{M}{Q}\right)**2*x**3*y**2

2*\left(\frac{M}{Q}\right)**2*x**2*y*(\left(\frac{M}{Q}\right)**2*x**2*y**2 + y - 1)

-(2*\left(\frac{M}{Q}\right)**2*x**2*y + 1)*(\left(\frac{M}{Q}\right)**2*x**2*y**2 + y - 1)

x*y**2*(2*\left(\frac{M}{Q}\right)**2*x**2*y + 1)

# Derive Projectors